In [ ]:
import boto3
import base64
import io
from PIL import Image
from botocore.client import Config
from s3_utilities import get_s3_connection_profile, S3ConnectionProfile

In [ ]:
s3_connection_profile = get_s3_connection_profile()
#s3_connection_profile = S3ConnectionProfile("s3://minio-api-minio.apps.ai-dev01.kni.syseng.devcluster.openshift.com", 
#                                 "edb-aidb", "us-east-1", "false", "minio", "minio1234!", "recommender_images")

In [ ]:
MAX_BASE64_LENGTH = 131072  # 128 KiB in characters

def downscale_image_until_small_enough(image_bytes):
    image = Image.open(io.BytesIO(image_bytes))

    if image.mode in ("RGBA", "P"):
        image = image.convert("RGB")

    buffer = io.BytesIO()
    image.save(buffer, format="JPEG", quality=85)
    base64_encoded = base64.b64encode(buffer.getvalue())

    was_resized = False

    while len(base64_encoded) > MAX_BASE64_LENGTH:
        was_resized = True
        width, height = image.size
        new_size = (int(width * 0.9), int(height * 0.9))
        image = image.resize(new_size, Image.Resampling.LANCZOS)
        buffer = io.BytesIO()
        image.save(buffer, format="JPEG", quality=85)
        base64_encoded = base64.b64encode(buffer.getvalue())

    return base64_encoded, was_resized, buffer

def get_base64_length_from_minio(bucket_name, prefix='', endpoint_url='', access_key='', secret_key=''):

    s3 = boto3.client(
        's3',
        endpoint_url=endpoint_url,
        aws_access_key_id=access_key,
        aws_secret_access_key=secret_key,
        config=Config(signature_version='s3v4'),
        region_name='us-east-1'
    )

    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

    if 'Contents' not in response:
        print("No objects found in the bucket.")
        return

    for obj in response['Contents']:
        key = obj['Key']
        if not key.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.webp')):
            continue

        print(f"Processing: {key}")
        image_obj = s3.get_object(Bucket=bucket_name, Key=key)
        image_bytes = image_obj['Body'].read()

        base64_encoded, was_resized, resized_buffer = downscale_image_until_small_enough(image_bytes)

        print(f"Final base64 length for {key}: {len(base64_encoded)} characters")

        if was_resized:
            print(f"Overwriting original image: {key}")
            s3.put_object(
                Bucket=bucket_name,
                Key=key,
                Body=resized_buffer.getvalue(),
                ContentType='image/jpeg'
            )
            print("Overwrite complete.\n")
        else:
            print("No resizing needed.\n")
        

# Example usage
get_base64_length_from_minio(
    bucket_name='edb-aidb',
    prefix='recommender_images/',
    endpoint_url=s3_connection_profile.endpoint_url,  # Replace with your MinIO endpoint
    access_key=s3_connection_profile.access_key,
    secret_key=s3_connection_profile.secret_key
)
